# Carga de datos

In [ ]:
import pandas as pd
# Carga
df = pd.read_csv('data/departamentos_clean.csv')
df.shape

## Modelo SVM

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn import svm

In [ ]:
# Para usar los métodos SVM las variables categoricas "comuna" y "estacion_cercana" se transforman a variables dummys
df = pd.get_dummies(df)

In [ ]:
# Separación de etiqueta Y y caracteristicas X
y = df.iloc[:,0]
X = df.iloc[:,1:]
# Subconjunto de entrenamiento y testing de 80 y 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

Prueba del modelo SVM por defecto: C=1, kernel='rbf', degree=3

In [ ]:
# Definición del modelo SVM
clf = svm.SVC()
# Entrenamiento del modelo
clf.fit(X_train, y_train)
# Precisión
clf.score(X_test, y_test)
# Predicción de X test
y_test_pred = clf.predict(X_test)

#### Cross Validation

In [ ]:
# Clasificación SVC por defecto
accuracy = cross_val_score(clf, X, y, cv=5)
print(accuracy)
# Precisión promedio
print("Precisión promedio: ", accuracy.mean() * 100)

Podemos observar que la precisión del modelo a través de *cross-validation* con k=10 es en promedio de 84.6%

In [ ]:
# Obtención de predicciones por medio de validación cruzada
y_pred = cross_val_predict(clf, X, y, cv=10)
# Matriz de confusión
confusion_matrix(y, y_pred)

#### f1 score

In [ ]:
f1_score(y_test, y_test_pred)

Si bien se observa la matriz de confusión puede clasificar perfectamente las preferencias 0, ha fallado el 100% de las preferencias 1, y tal como se conoce la naturaleza del problema, es más interesante poder clasificar correctamente la clase 1

#### auc score

In [ ]:
roc_auc_score(y_test, y_test_pred)

## Prueba automatica de diferentes modelos SVM

In [ ]:
for kernel in ['linear', 'poly', 'rbf', 'sigmoid'] :
    print('====> Kernel : ', kernel)
    for C in [0.01, 0.1, 1, 1.5, 3, 5, 10, 20, 50, 100] :
        print('==> C : ', C)
        clf_iter = svm.SVC(kernel=kernel, C=C)
        accuracy = cross_val_score(clf_iter, X, y, cv=5)
        print("Precisión promedio: ", accuracy.mean() * 100)
        clf_iter.fit(X_train, y_train)
        y_test_pred = clf_iter.predict(X_test)
        f1 = f1_score(y_test, y_test_pred, average='macro')
        confusion_matrix(y_test, y_test_pred)
        print('f1 : ', f1)
        aucscore = roc_auc_score(y_test, y_test_pred, average='macro')
        print('auc : ', aucscore)